In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

In [2]:
rootdir='../data/rawfiles/'

In [3]:
rainfall_data = []
i=0
f=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files[:]):
        rainfall_data.append(pd.read_excel(rootdir+'/'+file, 
                            index_col=None, 
                            header=None, 
                            sheet_name=None,
                            usecols='A:M'))

In [4]:
two_digit=[]
for i in tqdm(range(rainfall_data.__len__())):
    temp=dict(rainfall_data[i])
    n_temp={}
    for k in temp.keys():
        if len(str(k))==2:
            two_digit.append(k)
            n_temp['19'+k]=temp[k].copy()
        else:
            n_temp[k]=temp[k].copy()
    rainfall_data[i]=n_temp

In [5]:
# extracts data about the stations
all_stations = []
catchment_no = []
district = []
lat = []
lon = []
# i is the name of sheet (year), k is file number
for file in tqdm(rainfall_data):
    for sheet in file.keys(): 
        col = 0
        while col+4 < file[sheet].shape[0]: 
            catchment_no.append(file[sheet].iloc[col+1, 2])
            district.append(str(file[sheet].iloc[col, 7]).strip().lower())
            lat.append(file[sheet].iloc[col+1, 7])
            lon.append(file[sheet].iloc[col+1, 11])
            station_n=str(file[sheet].iloc[col, 2]).strip().lower()+'-'+district[-1]
            all_stations.append(station_n)
            col += 50

In [6]:
detail_stations = pd.DataFrame({
    'station': all_stations,
    'catchment-no': catchment_no,
    'district': district,
    'latitude': lat,
    'longitude': lon})

In [7]:
detail_stations.to_csv(r'../data/processedfiles/output_details_new.csv',index=False)

In [7]:
years=[year for year in range(1957,2013)]
year=[]
day=[]
days=[j for j in range(1,366)]
no_of_days=[]
sum_no_of_days=[0]
leap=[]
for i in tqdm(range(years.__len__())):
    day.extend(days)
    year.extend([years[i] for j in range(1,366)])
    if (years[i]%4==0 and years[i]%100!=0) or years[i]%400==0:
        day.append(366)
        leap.append(True)
        no_of_days.append(366)
        sum_no_of_days.append(sum_no_of_days[-1]+no_of_days[-1])
        year.append(years[i])
    else:
        leap.append(False)
        no_of_days.append(365)
        sum_no_of_days.append(sum_no_of_days[-1]+no_of_days[-1])

In [8]:
# creates a dataframe with value =-1 for all stations and all dates.
temp={key:[-1]*day.__len__() for key in all_stations}
rain_df=pd.DataFrame(temp)
rain_df.insert(0,'day-number',day)
rain_df.insert(0,'year',year)

In [9]:
rain_df

,year,day-number,jalore-jalore,sanchore-jalore,jaswantpura-jalore,bhinmal-jalore,ahore-jalore,bankli *-jalore,bhadragoan *-jalore,bithan *-jalore,...,kalingra-banswara,lohariya-banswara,kalinjjra-banswara,pilani (imd)-jhunjhunu,nawalgarh-jhunjhunu,malsesar-jhunjhunu,buhana-jhunjhunu,bhuhana-jhunjhunu,malsisar-jhunjhunu,malisar-jhunjhunu
0,1957,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1957,2,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1957,3,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1957,4,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1957,5,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20449,2012,362,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
20450,2012,363,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
20451,2012,364,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
20452,2012,365,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [10]:
# i=-year, j=row index, k=month no. start,end=starting row no and ending row no of the data
last=0
for df in tqdm(rainfall_data):
    last+=1
#     print('------------------------------------index',rainfall_data.index(df))
#     last=rainfall_data.index(df)
#     display('======================================'+str(last))
    for i in df.keys():
        j=0
        while j<=df[str(i)].shape[0]:
            start=sum_no_of_days[int(i)-1957]
#                 print(df[str(i)].iloc[j,2],i,j,'======================================'+str(last))
            for k in range(1,13):
                if k in [1,3,5,7,8,10,12]:
                    end=start+31-1
#                       print(rain_df.loc[start:end,str(df[str(i)].iloc[j,2]).upper()])
#                     print(type(df[str(i)].iloc[j+5:j+36,k].values))
#                     print(str(df[str(i)].iloc[j,2]).upper())
#                     rain_df.loc[start:end,str(df[str(i)].iloc[j,2]).upper()]=df[str(i)].iloc[j+5:j+36,k].values
                    t=df[str(i)].iloc[j+5:j+36,k].values
                elif k == 2:
                    if leap[int(i)-1957]:
                        end=start+29-1
                        t=df[str(i)].iloc[j+5:j+34,k].values
                    else:
                        end=start+28-1
                        t=df[str(i)].iloc[j+5:j+33,k].values
                else:
                    end=start+30-1
                    t=df[str(i)].iloc[j+5:j+35,k].values
                t[t=='N.A']='-1'
                t[t=='-']='-1'
                t[str(t)=='']='-1'
                t[t==np.NaN]='-1'
                t=np.array(list(map(lambda v: re.sub(r'\.\.','.', str(v)) ,t)))
                t=np.array(list(map(lambda v: re.sub(r'^[\D]*$','-1', str(v)) ,t)))
                t=np.array(list(map(lambda v: re.sub(r'[a-zA-Z]','', str(v)) ,t)))
                t=np.array(list(map(lambda v: re.sub(r'(\s+)','-1', str(v)) ,t)))
                try:
                    t=t.astype(float)
                    np.round(t,2)
                except:
                    print(type(str(t[1])),type(t[2]),t,df[str(i)],i,j)
#                     print(t.dtype)
                rain_df.loc[start:end,str(df[str(i)].iloc[j,2]).lower()+'-'+str(df[str(i)].iloc[j,7]).lower()]=t
                start=end+1
            j+=50
print('------------------------------'+str(last))


------------------------------43


In [11]:
rain_df.head()

,year,day-number,jalore-jalore,sanchore-jalore,jaswantpura-jalore,bhinmal-jalore,ahore-jalore,bankli *-jalore,bhadragoan *-jalore,bithan *-jalore,...,kishangarh -alwar,alwar -alwar,jayal -nagaur,chikhli -dungarpur,indergarh -kota,pokaran -jaisalmer,ramgarh -jaisalmer,surewala -ganganagar,sajjangarh -banswara,garhi -banswara
0,1957,1,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,1957,2,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
2,1957,3,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3,1957,4,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
4,1957,5,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


In [12]:
rain_df=rain_df.astype(str)
rain_df=rain_df.replace(to_replace='-',value='-1')

rain_df=rain_df.replace(to_replace=r'\.\.',value='.',regex=True)

rain_df=rain_df.replace(to_replace=r'([a-zA-Z])',value='',regex=True)

rain_df=rain_df.replace(to_replace=r'(\s+)',value='',regex=True)

rain_df=rain_df.replace(to_replace='',value='-1')

rain_df=rain_df.replace(to_replace=np.NaN,value='-1')

rain_df=rain_df.fillna('-1')

In [13]:
rain_df.head()

,year,day-number,jalore-jalore,sanchore-jalore,jaswantpura-jalore,bhinmal-jalore,ahore-jalore,bankli *-jalore,bhadragoan *-jalore,bithan *-jalore,...,kishangarh -alwar,alwar -alwar,jayal -nagaur,chikhli -dungarpur,indergarh -kota,pokaran -jaisalmer,ramgarh -jaisalmer,surewala -ganganagar,sajjangarh -banswara,garhi -banswara
0,1957,1,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
1,1957,2,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
2,1957,3,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
3,1957,4,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
4,1957,5,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1


In [14]:
rain_df.columns[2:2454]

Index(['jalore-jalore', 'sanchore-jalore', 'jaswantpura-jalore',
       'bhinmal-jalore', 'ahore-jalore', 'bankli *-jalore',
       'bhadragoan *-jalore', 'bithan *-jalore', 'bithana *-jalore',
       'bithan *-nan',
       ...
       'kishangarh -alwar', 'alwar -alwar', 'jayal -nagaur',
       'chikhli -dungarpur', 'indergarh -kota', 'pokaran -jaisalmer',
       'ramgarh -jaisalmer', 'surewala -ganganagar', 'sajjangarh -banswara',
       'garhi -banswara'],
      dtype='object', length=2452)

In [15]:
import datetime
rain_df['date']=rain_df.apply(lambda x:(datetime.datetime(int(x['year']),1,1)+datetime.timedelta(days=int(int(x['day-number'])-1))).strftime('%d/%m/%Y'),axis=1)
rain_df['date']=pd.to_datetime(rain_df['date'],format="%d/%m/%Y")
col=['year','day-number','date']
col.extend(list(rain_df.columns[2:2454]))
rain_df=rain_df[col]

In [19]:
rain_df.head(20)

,year,day-number,date,jalore-jalore,sanchore-jalore,jaswantpura-jalore,bhinmal-jalore,ahore-jalore,bankli *-jalore,bhadragoan *-jalore,...,kishangarh -alwar,alwar -alwar,jayal -nagaur,chikhli -dungarpur,indergarh -kota,pokaran -jaisalmer,ramgarh -jaisalmer,surewala -ganganagar,sajjangarh -banswara,garhi -banswara
0,1957,1,1957-01-01,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
1,1957,2,1957-01-02,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
2,1957,3,1957-01-03,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
3,1957,4,1957-01-04,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
4,1957,5,1957-01-05,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
5,1957,6,1957-01-06,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1
6,1957,7,1957-01-07,12.7,0.0,20.8,0.0,15.7,10.2,-1.0,...,-1,0.0,-1,-1,-1,0.0,13.2,-1,-1,-1
7,1957,8,1957-01-08,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,2.5,-1,-1,-1,0.0,0.0,-1,-1,-1
8,1957,9,1957-01-09,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,25.4,-1,-1,-1,12.7,0.0,-1,-1,-1
9,1957,10,1957-01-10,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-1,0.0,-1,-1,-1,0.0,0.0,-1,-1,-1


In [17]:
rain_df.to_csv('../data/processedfiles/rainfall-data-final.csv',index=False)

In [18]:
rain_df.shape

(20454, 2455)